## Rerun imports and initialization

In [1]:
from __future__ import annotations

import math
import uuid
from collections import namedtuple
from math import cos, sin

import numpy as np
import rerun as rr  # pip install rerun-sdk
import rerun.blueprint as rrb
from rerun.notebook import Viewer  # pip install rerun-notebook

DEV ENVIRONMENT DETECTED! Re-importing rerun from: /Users/aedm/w/rerun3/rerun_py/rerun_sdk


Starting asset server due to RERUN_NOTEBOOK_ASSET=serve-local
Loading assets into memory…
Serving rerun notebook assets at http://127.0.0.1:53509


## Helper to create the colored cube

This code exists in the `rerun.utilities` package, but is included here for context.

In [2]:
ColorGrid = namedtuple("ColorGrid", ["positions", "colors"])


def build_color_grid(x_count: int = 10, y_count: int = 10, z_count: int = 10, twist: float = 0) -> ColorGrid:
    """
    Create a cube of points with colors.

    The total point cloud will have x_count * y_count * z_count points.

    Parameters
    ----------
    x_count, y_count, z_count:
        Number of points in each dimension.
    twist:
        Angle to twist from bottom to top of the cube

    """

    grid = np.mgrid[
        slice(-x_count, x_count, x_count * 1j),
        slice(-y_count, y_count, y_count * 1j),
        slice(-z_count, z_count, z_count * 1j),
    ]

    angle = np.linspace(-float(twist) / 2, float(twist) / 2, z_count)
    for z in range(z_count):
        xv, yv, zv = grid[:, :, :, z]
        rot_xv = xv * cos(angle[z]) - yv * sin(angle[z])
        rot_yv = xv * sin(angle[z]) + yv * cos(angle[z])
        grid[:, :, :, z] = [rot_xv, rot_yv, zv]

    positions = np.vstack([xyz.ravel() for xyz in grid])

    colors = np.vstack([
        xyz.ravel()
        for xyz in np.mgrid[
            slice(0, 255, x_count * 1j),
            slice(0, 255, y_count * 1j),
            slice(0, 255, z_count * 1j),
        ]
    ])

    return ColorGrid(positions.T, colors.T.astype(np.uint8))

## Logging some data

Now we can log some data and add it to the recording, and display it using `notebook_show`.

Note that displaying a recording will consume the data, so it will not be available for use in a subsequent cell.

In [3]:
rr.auth()

Authorization URL: https://api.workos.com/user_management/authorize?client_id=client_01JZ3JVQW6JNVXME6HV9G4VR0H&redirect_uri=http%3A%2F%2F127.0.0.1%3A5001%2Flogin-callback&response_type=code&organization_id=org_01K6MTXXG0YSAFP6Q7AREM0DJQ&code_challenge=EETIdIsWEJmtddWLD05SJ0jceTmSMpdALo8nYcZq9KQ&code_challenge_method=S256


In [7]:
rr._auth.start_server(background=True, open_browser=False)
print("Server is ready to handle authentication requests!")

 * Serving Flask app 'rerun.auth'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2025 15:22:40] "GET /status HTTP/1.1" 200 -


✓ Authentication server started on http://localhost:5001
Waiting for server to start... 50 attempts left
Status response: 200
Server is ready to handle authentication requests!


127.0.0.1 - - [10/Nov/2025 15:23:10] "GET /login HTTP/1.1" 302 -


Code: 01K9Q2CJ9X14ZMVJAF410P0H9T


127.0.0.1 - - [10/Nov/2025 15:23:14] "GET /login-callback?code=01K9Q2CJ9X14ZMVJAF410P0H9T HTTP/1.1" 200 -


Auth response: <Response [200]>
Auth response JSON: {'user': {'object': 'user', 'id': 'user_01K2FDM6QGSX502AVK8JP1Q2AX', 'email': 'gabor@rerun.io', 'email_verified': True, 'first_name': 'Gábor', 'last_name': 'Gyebnár', 'profile_picture_url': None, 'last_sign_in_at': '2025-11-10T14:20:29.253Z', 'locale': 'en-US', 'created_at': '2025-08-12T15:15:49.520Z', 'updated_at': '2025-11-10T14:20:29.253Z', 'external_id': None}, 'organization_id': 'org_01K6MTXXG0YSAFP6Q7AREM0DJQ', 'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6InNzb19vaWRjX2tleV9wYWlyXzAxSlozSlZRTTFZUllZTkNSUUtIUERKUEZDIn0.eyJ1cm46cmVydW46ZW1haWwiOiJnYWJvckByZXJ1bi5pbyIsImlzcyI6Imh0dHBzOi8vYXBpLndvcmtvcy5jb20vdXNlcl9tYW5hZ2VtZW50L2NsaWVudF8wMUpaM0pWUVc2Sk5WWE1FNkhWOUc0VlIwSCIsInN1YiI6InVzZXJfMDFLMkZETTZRR1NYNTAyQVZLOEpQMVEyQVgiLCJzaWQiOiJzZXNzaW9uXzAxSzlRMkNKUTdLMVAwRDRBQkFSRlBSTjRGIiwianRpIjoiMDFLOVEyQ0pTUVk1S0taQ0cwUVIxMUtTN1QiLCJvcmdfaWQiOiJvcmdfMDFLNk1UWFhHMFlTQUZQNlE3QVJFTTBESlEiLCJyb2xlIjoiYWRtaW4iLCJyb2xlcyI6WyJhZG1pbiJdLCJwZX

127.0.0.1 - - [10/Nov/2025 15:23:16] "GET / HTTP/1.1" 200 -


In [9]:
rr._auth.display_login_link()

In [10]:
rr._auth.get_status()

{'authenticated': True,
 'profile': {'id': 'user_01K2FDM6QGSX502AVK8JP1Q2AX',
  'email': 'gabor@rerun.io',
  'first_name': 'Gábor',
  'last_name': 'Gyebnár'},
 'token': True}

In [11]:
rr._auth.access_token

'eyJhbGciOiJSUzI1NiIsImtpZCI6InNzb19vaWRjX2tleV9wYWlyXzAxSlozSlZRTTFZUllZTkNSUUtIUERKUEZDIn0.eyJ1cm46cmVydW46ZW1haWwiOiJnYWJvckByZXJ1bi5pbyIsImlzcyI6Imh0dHBzOi8vYXBpLndvcmtvcy5jb20vdXNlcl9tYW5hZ2VtZW50L2NsaWVudF8wMUpaM0pWUVc2Sk5WWE1FNkhWOUc0VlIwSCIsInN1YiI6InVzZXJfMDFLMkZETTZRR1NYNTAyQVZLOEpQMVEyQVgiLCJzaWQiOiJzZXNzaW9uXzAxSzlRMkNKUTdLMVAwRDRBQkFSRlBSTjRGIiwianRpIjoiMDFLOVEyQ0pTUVk1S0taQ0cwUVIxMUtTN1QiLCJvcmdfaWQiOiJvcmdfMDFLNk1UWFhHMFlTQUZQNlE3QVJFTTBESlEiLCJyb2xlIjoiYWRtaW4iLCJyb2xlcyI6WyJhZG1pbiJdLCJwZXJtaXNzaW9ucyI6WyJyZWFkLXdyaXRlIiwid2lkZ2V0czp1c2Vycy10YWJsZTptYW5hZ2UiXSwiZXhwIjoxNzYyODcwOTk0LCJpYXQiOjE3NjI3ODQ1OTR9.N4zm1R6WYrLnms7w5MFPQQhk0ClF5Q9vFJbmbg35frn0FFKLiIxCqcTsM2C4UcUYkbHONf6GpFbgeJWUi2Kvo8IXzyvIwSaFpZDpa1jt1mOlmBx9N5Xawi2colPLk6LXYjnszJrPYgVs59JmM8o8uLvt_0dgxrFS6Dv0AzdJ9r0AZwd9Mt0ernueK1plO4bJkpfHvCmn1DuCTgu9Ai67LCa1O1_hULe1IOXM-PCAehMUDoZu_LK1PWtgWsAWruYKBxDZbUd42JDiSz72UIijWJ0pWWhrsJsMj79dlvKSGp4cgpROXNY_VToZdJjwt9V8BmIku8DgPkMO0Dj7uZ02lQ'

In [3]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))

rr.notebook_show()

Viewer()

## Logging live data

Using `rr.notebook_show` like above buffers the data in the recording stream, but doesn't process it until the call to `rr.notebook_show`.

However, `rr.notebook_show` can be called at any time during your cell's execution to immediately display the Rerun viewer. You can then incrementally stream to it. Here we add a sleep to simulate a cell that does a lot more processing. By calling `notebook_show` first, we can see the output of our code live while it's still running.

In [ ]:
from time import sleep

rr.init("rerun_example_cube")

rr.notebook_show()

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    sleep(0.05)
    rr.set_time("step", sequence=t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))

## Incremental logging

Note that until we either reset the recording stream (by calling `rr.init()`), or create a new output widget (by calling `rr.notebook_show()` The currently active stream in the kernel will continue to send events to the existing widget.

The following will add a rotation to the above recording.

In [ ]:
for t in range(STEPS):
    sleep(0.01)
    rr.set_time("step", sequence=t)
    rr.log("cube", rr.Transform3D(rotation=rr.RotationAxisAngle(axis=[1, 0, 0], degrees=t)))

## Starting a new recording

You can always start another recording by calling `rr.init(...)` again to reset the global stream, or alternatively creating a separate recording stream using the `rr.RecordingStream` constructor (discussed more below)

In [ ]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    h_grid = build_color_grid(10, 3, 3, twist=twists[t])
    rr.log("h_grid", rr.Points3D(h_grid.positions, colors=h_grid.colors, radii=0.5))
    v_grid = build_color_grid(3, 3, 10, twist=twists[t])
    rr.log("v_grid", rr.Points3D(v_grid.positions, colors=v_grid.colors, radii=0.5))

rr.notebook_show()

## Adjusting the view

The  `notebook_show` method also lets you adjust properties such as width and height.

In [ ]:
rr.init("rerun_example_cube")

small_cube = build_color_grid(3, 3, 3, twist=0)
rr.log("small_cube", rr.Points3D(small_cube.positions, colors=small_cube.colors, radii=0.5))

rr.notebook_show(height=400)

To update the default width and height, you can use the `rerun.notebook.set_default_size` function.

Note that you do not need to initialize a recording to use this function.

In [ ]:
from rerun.notebook import set_default_size

set_default_size(width=400, height=400)

rr.init("rerun_example_cube")

small_cube = build_color_grid(3, 3, 3, twist=0)
rr.log("small_cube", rr.Points3D(small_cube.positions, colors=small_cube.colors, radii=0.5))

rr.notebook_show()

In [ ]:
set_default_size(width=640, height=480)

## Using blueprints

Rerun blueprints can be used with `rr.notebook_show()`

For example, we can split the two grids into their own respective views.

In [ ]:
rr.init("rerun_example_cube")

blueprint = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(name="Horizontal grid", origin="h_grid"),
        rrb.Spatial3DView(name="Vertical grid", origin="v_grid"),
        column_shares=[2, 1],
    ),
    collapse_panels=True,
)

rr.notebook_show(blueprint=blueprint)

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    h_grid = build_color_grid(10, 3, 3, twist=twists[t])
    rr.log("h_grid", rr.Points3D(h_grid.positions, colors=h_grid.colors, radii=0.5))
    v_grid = build_color_grid(3, 3, 10, twist=twists[t])
    rr.log("v_grid", rr.Points3D(v_grid.positions, colors=v_grid.colors, radii=0.5))

## Extra convenience

Rerun blueprints types also implement `_ipython_display_()` directly, so if a blueprint is the last element in your cell the right thing will happen.

Note that this mechanism only works when you are using the global recording stream.

In [ ]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    h_grid = build_color_grid(10, 3, 3, twist=twists[t])
    rr.log("h_grid", rr.Points3D(h_grid.positions, colors=h_grid.colors, radii=0.5))
    v_grid = build_color_grid(3, 3, 10, twist=twists[t])
    rr.log("v_grid", rr.Points3D(v_grid.positions, colors=v_grid.colors, radii=0.5))

rrb.Spatial3DView(name="Horizontal grid", origin="h_grid")

## Working with non-global streams

Sometimes it can be more explicit to work with specific (non-global recording) streams via `rr.RecordingStream` constructor.

In this case, remember to call `notebook_show` directly on the recording stream. As noted above, there is no way to use a bare Blueprint object in conjunction with a non-global recording.

In [ ]:
rec = rr.RecordingStream("rerun_example_cube_flat")

bp = rrb.Blueprint(collapse_panels=True)

rec.notebook_show(blueprint=bp)

flat_grid = build_color_grid(20, 20, 1, twist=0)
rec.log("flat_grid", rr.Points3D(flat_grid.positions, colors=flat_grid.colors, radii=0.5))

## Using the Viewer object directly

Instead of calling `notebook_show` you can alternatively hold onto the viewer object.

This lets you add additional recordings to a view.

In [ ]:
rec = rr.RecordingStream("rerun_example_multi_recording", recording_id=uuid.uuid4())

flat_grid = build_color_grid(20, 20, 1, twist=0)
rec.log("flat_grid", rr.Points3D(flat_grid.positions, colors=flat_grid.colors, radii=0.5))

viewer = Viewer(recording=rec, blueprint=rrb.Blueprint(rrb.BlueprintPanel(state="expanded")))
viewer.display()

In [ ]:
rec = rr.RecordingStream("rerun_example_multi_recording", recording_id=uuid.uuid4())

viewer.add_recording(rec)

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rec.set_time("step", sequence=t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rec.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))

## Controlling the Viewer

Other than sending a blueprint to the Viewer, some parts of it can also be controlled directly through Python.

In [ ]:
viewer = Viewer()
viewer.display()

recordings = [
    rr.RecordingStream("rerun_example_time_ctrl", recording_id="example_a"),
    rr.RecordingStream("rerun_example_time_ctrl", recording_id="example_b"),
]

rec_colors = {"example_a": [0, 255, 0], "example_b": [255, 0, 0]}

for rec in recordings:
    viewer.add_recording(rec)

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for rec in recordings:
    for t in range(STEPS):
        cube = build_color_grid(10, 10, 10, twist=twists[t])
        rec.set_time("step", sequence=t)
        rec.log("cube", rr.Points3D(cube.positions, colors=rec_colors[rec.get_recording_id()], radii=0.5))

The state of each panel in the viewer can be overridden, locking it in the specified state.

In [ ]:
viewer.update_panels(blueprint="expanded")

In multi-recording scenarios, the active recording can be set using `set_active_recording`. The timeline panel's state for the currently active recording can be controlled using `set_time_ctrl`.

In [ ]:
viewer.set_active_recording(recording_id="example_a")
viewer.set_time_ctrl(timeline="step", sequence=25)
viewer.set_active_recording(recording_id="example_b")
viewer.set_time_ctrl(timeline="step", sequence=75)

Switch between the two recordings in the blueprint panel to see the updated timelines.